In [2]:
import pandas as pd
import numpy as np
import copy
import csv
import itertools

from tqdm import tqdm
from Bio import SeqIO, SearchIO 

В Этом ноутбуке результаты прогона hmmscan'a по белочкам полученным из окрестности MftB по которому прогонялся изначальный blast 

In [18]:
file = '/home/marfa/Desktop/lab/mycofactocin/mftb_first_rodeo/hmm/mftb_first_rodeo_hmm_per_dom_tbl.txt'
res_domains = dict()
x = 10
count = 0
for res in SearchIO.parse(file, 'hmmscan3-domtab'):
        res_domains[res.id] = {}
        for hit in res.hits:
            if hit.id[:4] == 'TIGR':
                    call = hit.id
                    call += ',' + hit.description.split(":")[0]
            else:
                    call = hit.id
            for hsp in hit.hsps:
                overlap = False
                if res_domains[res.id].keys(): #not empty
                    names = copy.deepcopy(list(res_domains[res.id].keys()))
                    for name in names: #проверка на перекрытие с раннее добавленными генами 
                        for q_range in res_domains[res.id][name]:
                            if min(hsp.env_end, q_range[1]) - max(hsp.env_start, q_range[0]) > x:
                                overlap = True
                                break
                        if overlap:
                            break
                    if not overlap: 
                        if call.split(',')[-1] not in res_domains[res.id]:
                            res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                            
                        if (hsp.env_start, hsp.env_end) not in res_domains[res.id][call]:
                                        #для случая мультидоменного белка с одинаковыми доменами: 
                                        #один и тот же белок выравнивается на несколько мест
                            res_domains[res.id][call].append((hsp.env_start, hsp.env_end))
                       
                else:
                    res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
res_domains  

{'WP_162111891.1': {'TIGR01692,HIBADH': [(6, 253)]},
 'WP_029248820.1': {'Aldedh': [(15, 432)]},
 'WP_234898656.1': {'AMP-binding': [(0, 429)], 'AMP-binding_C': [(436, 512)]},
 'WP_019735257.1': {'KshA_C': [(115, 326)], 'Rieske': [(10, 106)]},
 'WP_162111893.1': {'TIGR03857,F420_MSMEG_2249': [(22, 350)]},
 'WP_085261796.1': {'TIGR03969,mycofactocin': [(6, 33)]},
 'WP_170306147.1': {'Mycofactocin_RRE': [(18, 98)]},
 'WP_211662799.1': {'TIGR03962,mycofact_rSAM': [(1, 342)]},
 'WP_065287205.1': {'TIGR03966,actino_HemFlav': [(2, 387)]},
 'WP_065287206.1': {'TIGR03964,mycofact_creat': [(6, 242)]},
 'WP_065287296.1': {'TIGR03965,mycofact_glyco': [(11, 471)]},
 'WP_065287207.1': {'TIGR03970,Rv0697': [(7, 474)]},
 'WP_162111895.1': {'COesterase': [(47, 545)]},
 'WP_065287210.1': {'DUF4442': [(19, 154)]}}

In [30]:
for ke, value in res_domains.items(): #сортирую по началу выравнивания гена на регион
    res_domains[ke] = {k: v for k, v in sorted(res_domains[ke].items(), key=lambda item: item[1][0][0])}

domains_names = {}

for key, value in res_domains.items():
    array = list(value.keys())
    tigr = []
    domain = []
    for key in array:
        if key.split(',')[0][:4] == 'TIGR':
            tigr.append(key.split(',')[0])
        else:
            tigr.append(np.nan)
        domain.append(key.split(',')[-1])        
    domains_names[key] = [tigr,domain]
domains = [x[-1] for x in domains_names.values()]
tigrs = [x[0] for x in domains_names.values()]

domains_names = pd.DataFrame({'domains': domains, 'tigr_id': tigrs}, index =res_domains.keys())
domains_names

,domains,tigr_id
WP_162111891.1,[HIBADH],[TIGR01692]
WP_029248820.1,[Aldedh],[nan]
WP_234898656.1,"[AMP-binding, AMP-binding_C]","[nan, nan]"
WP_019735257.1,"[Rieske, KshA_C]","[nan, nan]"
WP_162111893.1,[F420_MSMEG_2249],[TIGR03857]
WP_085261796.1,[mycofactocin],[TIGR03969]
WP_170306147.1,[Mycofactocin_RRE],[nan]
WP_211662799.1,[mycofact_rSAM],[TIGR03962]
WP_065287205.1,[actino_HemFlav],[TIGR03966]
WP_065287206.1,[mycofact_creat],[TIGR03964]
